# Example CIP API Usage

## The below functionality will allow you to authenticate in the CIP API with your credentials and programatically access the endpoints that are available to you

### This will load in the credentials you have exported in your shell (please see the README.md) or you can enter them here

In [2]:
import os
CIP_API_USERNAME = os.getenv("CIP_API_USERNAME") or "your_username_here"
CIP_API_PASSWORD = os.getenv("CIP_API_PASSWORD") or "your_password_here"
CIP_API_SERVER_URL = os.getenv("CIP_API_SERVER_URL") or "https://cip-api-server-url/api/{endpoint}"

### `get_url_json_response` below will return the json response from the url you provide (if successful) and will authenticate based on your credentials.

In [3]:
import requests


def get_authenticated_header():
    url = CIP_API_SERVER_URL
    auth_endpoint = "get-token/"

    irl_response = requests.post(
        url=url.format(endpoint=auth_endpoint),
        json=dict(
            username=CIP_API_USERNAME,
            password=CIP_API_PASSWORD,
        ),
    )
    irl_response_json = irl_response.json()
    token = irl_response_json.get('token')

    auth_header = {
        'Accept': 'application/json',
        "Authorization": "JWT {token}".format(token=token),
    }
    return auth_header

def get_url_json_response(url):
    response = requests.get(url=url, headers=get_authenticated_header())

    if response.status_code != 200:
        raise ValueError(
            "Received status: {status} for url: {url} with response: {response}".format(
            status=response.status_code, url=url, response=response.content)
        )
    return response.json()

### Create a string for the endpoint you wish to connect to
### A list of available endpoints can be viewed at https://cip-api-server-url/api/docs (replacing cip-api-server-url with the server you wish to connect to)

In [4]:
interpretation_request_list_endpoint = "interpretationRequestsList"

### Use `format` to construct the full URL

In [5]:
IR_list_url = CIP_API_SERVER_URL.format(endpoint=interpretation_request_list_endpoint)

### Pass this URL to `get_url_json_response` to get the json of all CIP API Interpretation Requests your credentials will allow you to view

In [6]:
IR_list = get_url_json_response(url=IR_list_url)

### Inspect the results

In [7]:
type(IR_list)

dict

In [8]:
IR_list.keys()

[u'count', u'previous', u'results', u'next']

In [9]:
type(IR_list['results'])

list

In [10]:
len(IR_list['results'])

100

In [20]:
IR_list = get_url_json_response(url=IR_list_url+'?page=11')

In [21]:
ir_id, ir_version = map(str, IR_list['results'][0]['interpretation_request_id'].split('-'))

In [22]:
ir_id, ir_version

('1011', '1')

In [27]:
IR_case_url = CIP_API_SERVER_URL.format(endpoint='interpretationRequests/{ir_id}/{ir_version}/')
IR_case_url = IR_case_url.format(ir_id=ir_id, ir_version=ir_version)
case_data = get_url_json_response(url=IR_case_url)

In [28]:
case_data

{u'case_priority': u'low',
 u'cip': u'nextcode',
 u'clinical_report': [{u'clinical_report_data': {u'candidateStructuralVariants': [],
    u'candidateVariants': [{u'additionalNumericVariantAnnotations': {},
      u'additionalTextualVariantAnnotations': {u'bamConfirmed': u'CONFIRMED',
       u'clinicalSignificance': u'VARIANT_OF_UNCERTAIN_SIGNIFICANCE',
       u'createdAt': u'2017-02-02 13:00:04.0',
       u'id': u'9efec370-9616-0134-4840-00505605089c',
       u'internalComment': u'Reviewed LR, PT  Insufficient evidence to classify as pathogenic or benign, therefore VUS. Note: no LOF reported in males with CDKL5 as thought to be lethal. Confirm and report back to patient.',
       u'isActive': u'1',
       u'oraRowscn': u'20614945',
       u'owner': u'Philip Twiss',
       u'pn': u'GEL_CSA-1011-1-LP2000711-DNA_D06_CBG',
       u'queryContext': u'{"chrom":"chrX","pos":"18646662","reference":"C","call":"T","report_mode":"candidate_genes","query_name":"GEL_tier_report/gel_tiers/all/AllVaria

In [33]:
interpreation_request_payload = case_data['interpretation_request_data']['json_request']
interpreation_request_payload
pedigree = interpreation_request_payload['pedigree']
tieredVariants = interpreation_request_payload['TieredVariants']
interpreted_genome = case_data['interpreted_genome'][-1]
clinical_report = case_data['clinical_report'][-1]

In [34]:
pedigree

{u'analysisPanels': [{u'multiple_genetic_origins': u'no',
   u'panelName': u'55940f0e22c1fc4f7d26e965',
   u'panelVersion': u'1.1',
   u'review_outcome': u'Ready for interpretation',
   u'specificDisease': u'Epileptic encephalopathy'}],
 u'gelFamilyId': u'FM50002048',
 u'participants': [{u'additionalInformation': {u'consent_opinion': u'Plated',
    u'qc_state': u'Passed medical review',
    u'relation_to_proband': u'Mother',
    u'yearOfBirth': u'1991'},
   u'adoptedStatus': u'not_adopted',
   u'affectionStatus': u'affected',
   u'ancestries': {u'chiSquare1KGenomesPhase3Pop': None,
    u'fathersEthnicOrigin': None,
    u'fathersOtherRelevantAncestry': None,
    u'mothersEthnicOrigin': None,
    u'mothersOtherRelevantAncestry': None},
   u'consanguineousParents': u'unknown',
   u'consanguineousPopulation': None,
   u'consentStatus': {u'carrierStatusConsent': False,
    u'primaryFindingConsent': True,
    u'programmeConsent': True,
    u'secondaryFindingConsent': False},
   u'dataModelCa

In [35]:
tieredVariants

[{u'additionalNumericVariantAnnotations': {u'EXAC_ALL': 4.119838e-05,
   u'EXAC_NFE': 7.495577e-05,
   u'UK10K_ALSPAC_ALL': 0.0007784120400000001,
   u'UK10K_TWINSUK_ALL': 0.0005393743},
  u'additionalTextualVariantAnnotations': {u'ConsequenceType': u'missense_variant',
   u'clinvar_accessions': u'',
   u'clinvar_clinicalSignificances': u'',
   u'clinvar_geneNames': u'',
   u'clinvar_reviewStatus': u'',
   u'clinvar_traits': u''},
  u'alternate': u'C',
  u'calledGenotypes': [{u'copyNumber': None,
    u'depthAlternate': None,
    u'depthReference': None,
    u'gelId': u'50001182',
    u'genotype': u'heterozygous',
    u'phaseSet': None,
    u'sampleId': u'LP2000711-DNA_D06'},
   {u'copyNumber': None,
    u'depthAlternate': None,
    u'depthReference': None,
    u'gelId': u'50001244',
    u'genotype': u'reference_homozygous',
    u'phaseSet': None,
    u'sampleId': u'LP2000712-DNA_B04'},
   {u'copyNumber': None,
    u'depthAlternate': None,
    u'depthReference': None,
    u'gelId': u'50

In [36]:
interpreted_genome

{u'cip_version': 1,
 u'created_at': u'2016-10-28T11:31:10.170224Z',
 u'gel_qc_outcome': [],
 u'interpreted_genome_data': {u'InterpretationRequestID': u'CSA-1011-1',
  u'analysisId': u'1',
  u'comments': None,
  u'companyName': u'WuXi NextCODE',
  u'referenceDatabasesVersions': {},
  u'reportURI': u'/gel/reference/LP2000711-DNA_D06/LP2000711-DNA_D06/CSA-1011-1/Alignments/',
  u'reportURL': u'https://gel.wuxinextcode.com/csa/studies/10188/cloned_studies',
  u'reportedStructuralVariants': None,
  u'reportedVariants': [{u'additionalNumericVariantAnnotations': {},
    u'additionalTextualVariantAnnotations': {u'hgvsc': u'ENST00000537107.1:c.185G>A',
     u'hgvsp': u'ENSP00000445688.1:p.Arg62Gln',
     u'refgene': u'NM_001145210',
     u'vepBiotype': u'protein_coding_KNOWN',
     u'vepConsequence': u'missense_variant',
     u'vepImpact': u'MODERATE'},
    u'alternate': u'T',
    u'calledGenotypes': [{u'copyNumber': None,
      u'depthAlternate': None,
      u'depthReference': None,
      u'ge

In [37]:
clinical_report

{u'clinical_report_data': {u'candidateStructuralVariants': [],
  u'candidateVariants': [{u'additionalNumericVariantAnnotations': {},
    u'additionalTextualVariantAnnotations': {u'bamConfirmed': u'CONFIRMED',
     u'clinicalSignificance': u'VARIANT_OF_UNCERTAIN_SIGNIFICANCE',
     u'createdAt': u'2017-02-02 13:00:04.0',
     u'id': u'9efec370-9616-0134-4840-00505605089c',
     u'internalComment': u'Reviewed LR, PT  Insufficient evidence to classify as pathogenic or benign, therefore VUS. Note: no LOF reported in males with CDKL5 as thought to be lethal. Confirm and report back to patient.',
     u'isActive': u'1',
     u'oraRowscn': u'20614945',
     u'owner': u'Philip Twiss',
     u'pn': u'GEL_CSA-1011-1-LP2000711-DNA_D06_CBG',
     u'queryContext': u'{"chrom":"chrX","pos":"18646662","reference":"C","call":"T","report_mode":"candidate_genes","query_name":"GEL_tier_report/gel_tiers/all/AllVariants","gene":"CDKL5","pn":"GEL_CSA-1011-1-LP2000711-DNA_D06_CBG","calculated_population_data_i

All of this is good, but do I have to keep parsing jsons eternally??!! Let's use the Models!

In [38]:
from protocols.reports_3_0_0 import *

In [39]:
ir_obj = InterpretationRequestRD.fromJsonDict(interpreation_request_payload)

In [40]:
pedigree = ir_obj.pedigree

In [41]:
pedigree

Ok! that easier. Let's get some values out of my data.

In [44]:
hpo_terms = []
for member in pedigree.participants:
    if member.hpoTermList:
        for hpo in member.hpoTermList:
            hpo_terms.append(hpo.term)
        

In [46]:
set(hpo_terms)

{u'HP:0001250', u'HP:0002349', u'HP:0012469'}

In [47]:
pedigree.analysisPanels

In [48]:
for analysis_panels in pedigree.analysisPanels:
    print analysis_panels.toJsonDict()

{u'panelVersion': u'1.1', u'review_outcome': u'Ready for interpretation', u'multiple_genetic_origins': u'no', u'specificDisease': u'Epileptic encephalopathy', u'panelName': u'55940f0e22c1fc4f7d26e965'}
